In [12]:
import pandas as pd
import numpy as np

In [102]:
#0. 'date' is the key for the rolling join. It does not have to be a date.
B = pd.DataFrame.from_dict(
    {'second_b':[0, 2, 4, 9],
     'wap_b': [10, 20, 30, 50]})
A = pd.DataFrame.from_dict(
    {'second_a':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
     'wap_a': -1})



In [100]:
A

,second_a,wap
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
5,5,-1
6,6,-1
7,7,-1
8,8,-1
9,9,-1


In [101]:
B

,second_b,wap
0,0,10
1,2,20
2,4,30
3,9,50


In [96]:
#1. Sort the table A and and B each by key.
A = A.sort_values('second')
B = B.sort_values('second')

#2. Add a column tag to A which are all 0 and a column tag to B that are all 1.
A['tag'] = 0
B['tag'] = 1

In [97]:
#3. Delete all columns except the key and tagfrom B (can be omitted, but it is clearer this way) and call the table B'. Keep B as an original - we are going to need it later.
B_ = B[['second','tag']] # You need two [], because you get a series otherwise.

#4. Concatenate A with B' to C and ignore the fact that the rows from B' has many NAs.
C = pd.concat([A, B_])

#5. Sort C by key.
C = C.sort_values('second')

In [98]:
C

,second,wap,tag
0,0,-1.0,0
0,0,NaN,1
1,1,-1.0,0
2,2,-1.0,0
1,2,NaN,1
3,3,-1.0,0
4,4,-1.0,0
2,4,NaN,1
5,5,-1.0,0
6,6,-1.0,0


In [67]:
#6. Make a new cumsum column with C = C.assign(groupNr = np.cumsum(C.tag))
C = C.assign(groupNr = np.cumsum(C.tag))

In [68]:
C

,second,wap,tag,groupNr
0,0,-1.0,0,0
0,0,NaN,1,1
1,1,-1.0,0,1
2,2,-1.0,0,1
1,2,NaN,1,2
3,3,-1.0,0,2
4,4,-1.0,0,2
2,4,NaN,1,3
5,5,-1.0,0,3
6,6,-1.0,0,3


In [69]:
#7. Using filtering (query) on tag get rid of all B'-rows.
C = C[C.tag == 0]

In [70]:
C

,second,wap,tag,groupNr
0,0,-1.0,0,0
1,1,-1.0,0,1
2,2,-1.0,0,1
3,3,-1.0,0,2
4,4,-1.0,0,2
5,5,-1.0,0,3
6,6,-1.0,0,3
7,7,-1.0,0,3
8,8,-1.0,0,3
9,9,-1.0,0,3


In [89]:
#8. Add a running counter column groupNr to the original B (integers from 0 to N-1 or from 1 to N, depending on whether you want forward or backward rolling join).
# B['groupNr'] = range(len(B)+1)[0:-1] # B's values are carried forward to A's values
B['groupNr'] = range(len(B))       # B's values are carried backward to A's values

In [90]:
B

,second,wap,tag,groupNr
0,0,10,1,0
1,2,20,1,1
2,4,30,1,2
3,9,50,1,3


In [91]:
#9. Join B with C on groupNr to D.
D = C.set_index('groupNr').join(B.set_index('groupNr'), lsuffix='_A', rsuffix='_B')

In [92]:
D

,second_A,wap_A,tag_A,second_B,wap_B,tag_B
groupNr,,,,,,
0,0,-1.0,0,0.0,10.0,1.0
1,1,-1.0,0,2.0,20.0,1.0
1,2,-1.0,0,2.0,20.0,1.0
2,3,-1.0,0,4.0,30.0,1.0
2,4,-1.0,0,4.0,30.0,1.0
3,5,-1.0,0,9.0,50.0,1.0
3,6,-1.0,0,9.0,50.0,1.0
3,7,-1.0,0,9.0,50.0,1.0
3,8,-1.0,0,9.0,50.0,1.0


In [83]:
[A, B]

[    second  wap  tag
 0        0   -1    0
 1        1   -1    0
 2        2   -1    0
 3        3   -1    0
 4        4   -1    0
 5        5   -1    0
 6        6   -1    0
 7        7   -1    0
 8        8   -1    0
 9        9   -1    0
 10      10   -1    0,
    second  wap  tag  groupNr
 0       0   10    1        1
 1       2   20    1        2
 2       4   30    1        3
 3       9   50    1        4]

# Rolling join

In [109]:
# initial table
B = pd.DataFrame.from_dict(
    {'second_b':[0, 2, 4, 9],
     'wap_b': [10, 20, 30, 50]})

# resulting table
A = pd.DataFrame.from_dict(
    {'second_a':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})



In [111]:
result = pd.merge_asof(A,
          B,
          left_on='second_a',
          right_on='second_b')

# Ordering for easier comparison
result = result[['second_a','wap_b']]
result

,second_a,wap_b
0,0,10
1,1,10
2,2,20
3,3,20
4,4,30
5,5,30
6,6,30
7,7,30
8,8,30
9,9,50


In [112]:
[A, B]

[    second_a
 0          0
 1          1
 2          2
 3          3
 4          4
 5          5
 6          6
 7          7
 8          8
 9          9
 10        10,
    second_b  wap_b
 0         0     10
 1         2     20
 2         4     30
 3         9     50]

## Rolling join with time id

In [122]:
# initial table
B = pd.DataFrame.from_dict(
    {'time_id':[1, 1, 1, 1, 2, 2, 2, 2],
     'second_b':[0, 2, 4, 9, 0, 1, 2, 5],
     'wap_b': [10, 20, 30, 50, 60, 70, 80, 90]})

# resulting table
A = pd.DataFrame.from_dict(
    {'time_id':np.repeat([1,2],11),
     'second_a':np.tile([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],2)})



In [131]:
B = B.sort_values('second_b')
A = A.sort_values('second_a')

In [139]:
result1 = pd.merge_asof(A,
          B,
          left_on='second_a',
          right_on='second_b',
          by = 'time_id')

# Ordering for easier comparison
result1 = result1[['time_id', 'second_a','wap_b']]
result1 = result1.sort_values(['time_id','second_a']).reset_index(drop=False)
result1

,index,time_id,second_a,wap_b
0,0,1,0,10
1,2,1,1,10
2,4,1,2,20
3,6,1,3,20
4,8,1,4,30
5,11,1,5,30
6,12,1,6,30
7,14,1,7,30
8,17,1,8,30
9,19,1,9,50


In [137]:
B.sort_values(['time_id','second_b'])

,time_id,second_b,wap_b
0,1,0,10
1,1,2,20
2,1,4,30
3,1,9,50
4,2,0,60
5,2,1,70
6,2,2,80
7,2,5,90
